In [ ]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
from tqdm.notebook import tqdm

import shutil

In [ ]:
meta = pd.read_csv('mass_case_description_test_set.csv')
meta

In [ ]:
meta['id'] = meta['image file path'].apply(lambda x: x.split('/')[0])


In [ ]:
test.to_csv('test_aug_cibs_ddsm1.csv')

### Convert image to png and storage

In [ ]:
def get_all_files_in_folder(folder_path):
    # print(folder_path)
    all_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    return all_files

In [ ]:
# kiểm tra coi folder nào có 4 ảnh 
for i in meta['image file path']:
    name = i.split('/')[0]
    for x in range(1, 10):
        if os.path.exists(f'E:/Test-Mass-Roi/manifest-OlX3eVq41551709337535597984/CBIS-DDSM/{name}_{x}/'):
            ls =  len(get_all_files_in_folder(f'E:/Test-Mass-Roi/manifest-OlX3eVq41551709337535597984/CBIS-DDSM/{name}_{x}/'))
            # print(ls)
            if ls == 1:
                print('++++++++++++++++++++++++++++')
                print(ls)
                print(f'E:/Test-Mass-Roi/manifest-OlX3eVq41551709337535597984/CBIS-DDSM/{name}_{x}/')
            if ls > 2:
                print('++++++++++++++++++++++++++++')
                print(ls)
                print(f'E:/Test-Mass-Roi/manifest-OlX3eVq41551709337535597984/CBIS-DDSM{name}_{x}/')

In [ ]:
def procees_dcm(path):
    
    dicom = pydicom.dcmread(path)
    # try : 
    img = dicom.pixel_array
        
    img = apply_windowing(img, dicom)
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":  
        img = 1 - img
    image = np.uint8(img * 255)
    return image

In [ ]:
# Lấy tất cả các file trong ảnh full
def get_image(path_name, path_name_mask, path_origin, path_origin_mask):
    # df = pd.DataFrame(columns=['Image', 'Crop', 'Mask'])
    if os.path.exists('Test/Meta_data_test/') ==False: os.makedirs('Test/Meta_data_test/')
    if os.path.exists('Test/Roi_test/') ==False: os.makedirs('Test/Roi_test/')
    if os.path.exists('Test/Image_test/') ==False: os.makedirs('Test/Image_test/')
    if os.path.exists('Test/Mask_test/') ==False: os.makedirs('Test/Mask_test/')
    paths = get_all_files_in_folder(f'{path_origin}/{path_name}')
    # processing img full 
    if len(paths) > 1:
        print(f'{path_name} has 2 image in img full')
    sz_img = 0
    for path in paths:
        if os.path.exists(f'Test/Meta_data_test/{path_name}/{path_name}.png') : 
            sz_img = cv2.imread(f'Test/Meta_data_test/{path_name}/{path_name}.png').shape
            continue
        # if os.path.exists(f'Test/Meta_data_test/{path_name}/') == False: os.makedirs(f'Test/Meta_data_test/{path_name}/')
        img = procees_dcm(path)
        sz_img = img.shape
        imageio.imwrite(f'Test/Meta_data_test/{path_name}.png', img)
        # imageio.imwrite(f'Test/Image_test/{path_name}.png', img)
    # Processing image mask
    paths_mask = get_all_files_in_folder(f'{path_origin_mask}/{path_name_mask}')
    print(f'{path_origin_mask}/{path_name_mask}')
    print(len(paths_mask))
    print(paths_mask)
    for mask in paths_mask:
        img = procees_dcm(mask)
        path_mask = f'Test/Meta_data_test/{path_name}/{path_name_mask}_mask.png'
        path_roi = f'Test/Meta_data_test/{path_name}/{path_name_mask}_roi.png'
        # print(sz_img ,  img)
        # print(sz_img , img.shape)
        if sz_img == img.shape or (sz_img[0] == img.shape[0] and sz_img[1] == img.shape[1] ):
            # print(f'Mask/{path_name_mask}_mask.png')
            imageio.imwrite(path_mask, img)
            imageio.imwrite(f'Test/Mask_test/{path_name_mask}_mask.png', img)
        else:
            # print(f'Roi/{path_name_mask}_roi.png')
            imageio.imwrite(path_roi, img)
            imageio.imwrite(f'Test/Roi_test/{path_name_mask}_roi.png', img)
    
    
        

In [ ]:
from tqdm.notebook import tqdm


In [ ]:
for i in tqdm(meta.index):
    path_name = meta['image file path'][i].split('/')[0]
    path_name_mask = meta['ROI mask file path'][i].split('/')[0]
    # print(path_name_mask) E:/Processing Data\Test-Mass-Roi\manifest-OlX3eVq41551709337535597984\CBIS-DDSM
    path_origin = 'Test-Mass/manifest-KlyiOp8o8257600061251652132/CBIS-DDSM'
    path_origin_mask = 'Test-Mass-Roi/manifest-OlX3eVq41551709337535597984/CBIS-DDSM'
    get_image(path_name=path_name, path_name_mask=path_name_mask, path_origin=path_origin, path_origin_mask =path_origin_mask)

In [ ]:
meta['Image'] = meta['image file path'].apply(lambda x: x.split('/')[0])

In [ ]:
meta.pathology.value_counts()

In [ ]:
cag = {'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 0, 'MALIGNANT':1}

In [ ]:
meta['status'] = meta.pathology.apply(lambda x: cag[x])

In [ ]:
meta = meta[['Image', 'status']]
meta

### Combine alls in to mask

In [ ]:
meta['id_image'] = meta['image file path'].apply(lambda x: x.split('/')[0])

In [ ]:
image = np.unique(meta['id_image'])

In [ ]:
len(image)

In [ ]:
image_path = r'E:\Processing Data\Meta_data\Mass-Training_P_00001_LEFT_CC\Mass-Training_P_00001_LEFT_CC_1_mask.png'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Create a new blank image with the same size
blank_image = original_image.copy()

# Fill the blank image with zeros (black)
blank_image.fill(0)
plt.imshow(blank_image)

In [ ]:
blank_image =  cv2.cvtColor(blank_image, cv2.COLOR_GRAY2RGB)

In [ ]:
image1 = r'E:\Processing Data\Meta_data\Mass-Training_P_00001_LEFT_CC\Mass-Training_P_00001_LEFT_CC_1_mask.png'
img = cv2.imread(image1)
plt.imshow(blank_image + img)

In [ ]:
data = []
'''
Make sure all mask is true 
'''
for i in tqdm(image):
    paths = glob.glob(f'Meta_data_test/{i}/*')
    original_image = cv2.imread(f'Meta_data_test/{i}/{i}.png')
    blank_image = original_image.copy()
    blank_image.fill(0)
    if len(blank_image.shape) < 3: blank_image =  cv2.cvtColor(blank_image, cv2.COLOR_GRAY2RGB)
    # print(blank_image.shape)
    cnt = 0
    # print(paths)
    for path in paths:
        if 'mask' in path:
            cnt+=1
            img = cv2.imread(path)
            blank_image += img
    # print(cnt)
    if cnt > 0:
        print(f'Meta_data_test/{i}/{i}.png')
        data.append([f'Meta_data_test/{i}/{i}.png', f'Meta_data_test/{i}/{i}_mask.png'])
        imageio.imwrite(f'Meta_data_test/{i}/{i}_mask.png', blank_image)

In [ ]:
meta_seg = pd.DataFrame(columns=['Image', 'Mask'], data=data)

In [ ]:
meta_seg

In [ ]:
meta_seg.to_csv('meta_seg_test.csv')

In [ ]:
meta_seg = pd.read_csv('meta_seg_test.csv')

In [ ]:
meta_seg

In [ ]:
for path in meta_seg.Mask:
    img = cv2.imread(path)
    name = os.path.basename(path)
    imageio.imwrite(f'E:/Processing Data/Mask_test_full/{name}', img)